In [ ]:
# Exploration of our Number of Times Contact Attempted & Number of Times Reached variables' missing values. 

# Exploration: is our data on the number of times contact was attempted randomly missing?
# & is data on the number of times someone was reached randomly missing?
# How to check: are there statistically significant differences between features, if we split our data on missing/non-missing?

# (1) Number of Times Contact Attempted feature: 
# Create dummies to separate out noTimesContactAttempted missing / non-missing observations
model2c = data
model2c['noTimesContAtt_incl'] = data.noTimesContactAttempted.notnull()==True
model2c['noTimesReached_incl'] = data.noTimesReached.notnull()==True
# model2c.head()

# Import ttest from SciPy
from scipy.stats import ttest_ind

# Define the 2 groups to be compared for number of times contact attempted
noTCA_1 = model2c[model2c['noTimesContAtt_incl']==True]
noTCA_2 = model2c[model2c['noTimesContAtt_incl']==False]

# Run a t-test on verified (even though it's a dummy, we can look at the means to gauge if more reports were verif or not)

# Define a function that runs ttests on a couple different features to identify any stat signif differences
def tt_feat1(features):
    print
    print "t-test results for number of times contact attempted"
    # Loop through the list of features we want to test for stat signif differences between
    for x in features: 
        print 
        print x
        print 
        print ttest_ind(noTCA_1[x], noTCA_2[x])
        
tt_feat1(['actionTaken','urgent', 'actionable', 'closed', 'urgent', \
         'verified', 'phoneNo_incl', 'locAccurFixer_incl'])      

# (2) Number of Times Contact Reached: 
# Define the 2 groups to be compared for number of times reached
noTR_1 = model2c[model2c['noTimesReached_incl']==True]
noTR_2 = model2c[model2c['noTimesReached_incl']==False]

def tt_feat2(features):
    print
    print "t-test results for number of times reached:"
    for x in features: 
        print 
        print x
        print 
        print ttest_ind(noTR_1[x], noTR_2[x])    
        
tt_feat2(['actionTaken','urgent', 'actionable', 'closed', 'urgent', \
         'verified', 'phoneNo_incl', 'locAccurFixer_incl'])

# Next steps:
# Use average
# then can try distribution 

In [ ]:
# TO UPDATE: CREATE A CLASS SO THAT BALANCED_M1DATA CAN BE MODIFIED AND USED BY THE BAL_MODELLR FUNCTION! 
# def b_dataset:
#     # Note that we should have 1634 observations for this set.
#     neg = model1_data[model1_data['actionTaken'] == 0]
#     print neg.info()

#     # Sample 401 observations of this subsample. We select 401 to match the number of reports where action was taken. 
#     neg_samp = neg.sample(401)

#     # Construct the positive (action taken on reports) subsample. 
#     pos = model1_data[model1_data['actionTaken'] == 1]

#     # Combine both samples: sampled no action taken reports, entire set of action taken reports.
#     balanced_m1data = pd.concat([neg_samp,pos])
#     balanced_m1data.info()

In [ ]:
# This is old code used to create a training dataset. This 

# Note that only about 20% of our reports had action taken on them. 
# To make sure that our model can accurately predict reports where action was taken, we want to create a more balanced dataset.
# We create one that is 50/50 

# As a first step - separate out the data into cases where reports had action taken (a/t), or no a/t. 

# Create a variable that stores cases where action is not taken on a report.
# Note that we should have 1634 observations for this set.
neg = model1_data[model1_data['actionTaken'] == 0]
neg.info()

# Sample 401 observations of this subsample. We select 401 to match the number of reports where action was taken. 
neg_samp = neg.sample(401)

# Construct the positive (action taken on reports) subsample. 
pos = model1_data[model1_data['actionTaken'] == 1]

# Combine both samples: sampled no action taken reports, entire set of action taken reports.
balanced_m1data = pd.concat([neg_samp,pos])
balanced_m1data.info()

In [ ]:
# Define logistic regression model
model = LogisticRegression(C=1)

# Create a function to test out our features using a logistic regression model.
# This will allow us to more quickly add features and trims the amount of code we need to use each time. 
def bal_modelLR(baldf, modeldf, model):
    # Create our training & test datasets 
    for train,test in ShuffleSplit(len(baldf), n_iter=1, test_size=0.25,random_state=0):
        # Drop label from training data
        data_train = baldf.iloc[train].drop(['actionTaken'], axis=1)
        # Create label set 
        label_train = baldf['actionTaken'].iloc[train]
        # Define our test data (features), label. Note we have to drop malignant since we're generating this off the orig data. 
        data_test = modeldf.iloc[test].drop(['actionTaken'], axis=1)
        label_test = modeldf['actionTaken'].iloc[test]

    # Run logistic regression on our training data
    model_lr = model.fit(data_train, label_train)

    # Print out details on precision, recall for our model. 
    print classification_report(label_test,model_lr.predict(data_test))
    
    # Print out the coefficients
    model.coef[0]
    print zip(model.coef_[0],modeldf.drop('actionTaken',axis=1))

bal_modelLR(balanced_m1data, model1_data,model)


In [ ]:
# This is old code used to run the first logistic regression model.
# Replaced this with a function, because we're essentially running the same code each time we build a model.
# Using a function makes the code much easier to follow as long as the function is well-explained / commented. 

# Create our training & test datasets 
for train,test in ShuffleSplit(len(balanced_m1data), n_iter=1, test_size=0.25,random_state=0):
    # Drop label from training data
    data_train = balanced_m1data.iloc[train].drop(['actionTaken'], axis=1)
    # Create label set 
    label_train = balanced_m1data['actionTaken'].iloc[train]
    # Define our test data (features), label. Note we have to drop malignant since we're generating this off the orig data. 
    data_test = model1_data.iloc[test].drop(['actionTaken'], axis=1)
    label_test = model1_data['actionTaken'].iloc[test]

# Run logistic regression on our training data
model_lr1 = model.fit(data_train, label_train)

# Print out details on precision, recall for our model. 
print classification_report(label_test,model_lr1.predict(data_test))

In [ ]:
# Old code for model 2. Replaced by a function.

# Create our training & test datasets 
for train,test in ShuffleSplit(len(balanced_m2data), n_iter=1, test_size=0.25,random_state=0):
    # Drop label from training data
    data_train2 = balanced_m2data.iloc[train].drop(['actionTaken'], axis=1)
    # Create label set 
    label_train2 = balanced_m2data['actionTaken'].iloc[train]
    # Define our test data (features), label. Note we have to drop malignant since we're generating this off the orig data. 
    data_test2 = model2a.iloc[test].drop(['actionTaken'], axis=1)
    label_test2 = model2a['actionTaken'].iloc[test]
    
# Run logistic regression on our training data
model_lr2 = model2.fit(data_train2, label_train2)

# Print out details on precision, recall for our model. 
print classification_report(label_test2,model_lr2.predict(data_test2))